In [65]:
import pandas as pd
import numpy as np

In [66]:
ratings_df = pd.read_csv("Dataset\last\XWines_Slim_150K_ratings.csv") 
display(ratings_df.head(10))

C:\Users\antoi\AppData\Local\Temp\ipykernel_56616\2223713249.py:1: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  ratings_df = pd.read_csv("Dataset\last\XWines_Slim_150K_ratings.csv")


,RatingID,UserID,WineID,Vintage,Rating,Date
0,143,1356810,103471,1950,4.5,2021-11-02 20:52:59
1,199,1173759,111415,1951,5.0,2015-08-20 17:46:26
2,348,1164877,111395,1952,5.0,2020-11-13 05:40:26
3,374,1207665,111433,1953,5.0,2017-05-05 06:44:13
4,834,1075841,111431,1955,5.0,2016-09-14 20:18:38
5,876,1211463,111395,1955,5.0,2021-12-02 23:12:49
6,1005,1076348,111433,1955,4.5,2021-06-19 19:53:56
7,1020,1147051,111429,1955,5.0,2018-07-08 20:09:46
8,1029,1225931,111431,1955,5.0,2017-04-24 01:41:52
9,1399,1197513,111415,1958,5.0,2014-07-04 01:07:16


In [67]:
wines_df = pd.read_csv("Dataset\last\XWines_Slim_1K_wines.csv", index_col="WineID")
wines_df['WineID'] = wines_df.index
display(wines_df.head(3))

,WineName,Type,Elaborate,Grapes,Harmonize,ABV,Body,Acidity,Code,Country,RegionID,RegionName,WineryID,WineryName,Website,Vintages,WineID
WineID,,,,,,,,,,,,,,,,,
100001,Espumante Moscatel,Sparkling,Varietal/100%,['Muscat/Moscato'],"['Pork', 'Rich Fish', 'Shellfish']",7.5,Medium-bodied,High,BR,Brazil,1001,Serra Gaúcha,10001,Casa Perini,http://www.vinicolaperini.com.br,"[2020, 2019, 2018, 2017, 2016, 2015, 2014, 201...",100001
100002,Ancellotta,Red,Varietal/100%,['Ancellotta'],"['Beef', 'Barbecue', 'Codfish', 'Pasta', 'Pizz...",12.0,Medium-bodied,Medium,BR,Brazil,1001,Serra Gaúcha,10001,Casa Perini,http://www.vinicolaperini.com.br,"[2016, 2015, 2014, 2013, 2012, 2011, 2010, 200...",100002
100003,Cabernet Sauvignon,Red,Varietal/100%,['Cabernet Sauvignon'],"['Beef', 'Lamb', 'Poultry']",12.0,Full-bodied,High,BR,Brazil,1001,Serra Gaúcha,10002,Castellamare,https://www.emporiocastellamare.com.br,"[2021, 2020, 2019, 2018, 2017, 2016, 2015, 201...",100003


In [68]:

body_mapping = {
    'Very light-bodied': 1,
    'Light-bodied': 2,
    'Medium-bodied': 3,
    'Full-bodied': 4,
    'Very full-bodied': 5
}
acidity_mapping = {
    'Low': 1,
    'Medium': 2,
    'High': 3
}

type_mapping = {
    'White': 1,
    'Rosé': 2,
    'Red': 3,
    'Sparkling': 4,
    'Dessert': 5,
    'Dessert/Port': 6
}



# Apply the mappings to the DataFrame
wines_df['Body'] = wines_df['Body'].map(body_mapping)
wines_df['Acidity'] = wines_df['Acidity'].map(acidity_mapping)
wines_df['Type'] = wines_df['Type'].map(type_mapping)
display(wines_df.head(3))


,WineName,Type,Elaborate,Grapes,Harmonize,ABV,Body,Acidity,Code,Country,RegionID,RegionName,WineryID,WineryName,Website,Vintages,WineID
WineID,,,,,,,,,,,,,,,,,
100001,Espumante Moscatel,4,Varietal/100%,['Muscat/Moscato'],"['Pork', 'Rich Fish', 'Shellfish']",7.5,3,3,BR,Brazil,1001,Serra Gaúcha,10001,Casa Perini,http://www.vinicolaperini.com.br,"[2020, 2019, 2018, 2017, 2016, 2015, 2014, 201...",100001
100002,Ancellotta,3,Varietal/100%,['Ancellotta'],"['Beef', 'Barbecue', 'Codfish', 'Pasta', 'Pizz...",12.0,3,2,BR,Brazil,1001,Serra Gaúcha,10001,Casa Perini,http://www.vinicolaperini.com.br,"[2016, 2015, 2014, 2013, 2012, 2011, 2010, 200...",100002
100003,Cabernet Sauvignon,3,Varietal/100%,['Cabernet Sauvignon'],"['Beef', 'Lamb', 'Poultry']",12.0,4,3,BR,Brazil,1001,Serra Gaúcha,10002,Castellamare,https://www.emporiocastellamare.com.br,"[2021, 2020, 2019, 2018, 2017, 2016, 2015, 201...",100003


In [69]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import OneHotEncoder

wine_features = wines_df[['Type', 'Grapes', 'Harmonize', 'ABV', 'Body', 'Acidity', 'Vintages']]

# Text Vectorization - For features like harmonize
tfidf = TfidfVectorizer(stop_words='english')
wine_tfidf = tfidf.fit_transform(wine_features['Harmonize'].fillna(''))


# One-Hot Encoding for categorical features like 'Type'
one_hot_encoder = OneHotEncoder()
wine_one_hot = one_hot_encoder.fit_transform(wine_features[['Type', 'Body', 'Acidity', 'ABV']])

# Combining features (TF-IDF + OneHot)
import scipy.sparse as sp
wine_features_combined = sp.hstack([wine_tfidf, wine_one_hot])

cosine_sim = cosine_similarity(wine_features_combined, wine_features_combined)




In [70]:
wines_df["Harmonize"]=  wines_df["Harmonize"].apply(eval)

In [71]:
wines_df["Harmonize"]

WineID
100001                         [Pork, Rich Fish, Shellfish]
100002      [Beef, Barbecue, Codfish, Pasta, Pizza, Cheese]
100003                                [Beef, Lamb, Poultry]
100005                     [Beef, Lamb, Game Meat, Poultry]
100007                         [Pork, Rich Fish, Shellfish]
                                ...                        
199408    [Pasta, Shellfish, Maturated Cheese, Hard Chee...
199481                              [Beef, Lamb, Game Meat]
199533               [Pork, Rich Fish, Vegetarian, Poultry]
199885    [Pork, Vegetarian, Poultry, Lean Fish, Cured M...
200139    [Beef, Lamb, Spicy Food, Maturated Cheese, Har...
Name: Harmonize, Length: 1007, dtype: object

In [78]:
from sklearn.preprocessing import MultiLabelBinarizer
mlb = MultiLabelBinarizer()

onehot = pd.DataFrame(mlb.fit_transform(wines_df['Harmonize']), columns=mlb.classes_, index=wines_df.index)

wines_df = pd.concat([wines_df, onehot], axis=1)

In [74]:
wines_df

,WineName,Type,Elaborate,Grapes,Harmonize,ABV,Body,Acidity,Code,Country,...,Seafood,Shellfish,Snack,Soft Cheese,Soufflé,Spicy Food,Sweet Dessert,Tomato Dishes,Veal,Vegetarian
WineID,,,,,,,,,,,,,,,,,,,,,
100001,Espumante Moscatel,4,Varietal/100%,['Muscat/Moscato'],"[Pork, Rich Fish, Shellfish]",7.5,3,3,BR,Brazil,...,0,1,0,0,0,0,0,0,0,0
100002,Ancellotta,3,Varietal/100%,['Ancellotta'],"[Beef, Barbecue, Codfish, Pasta, Pizza, Cheese]",12.0,3,2,BR,Brazil,...,0,0,0,0,0,0,0,0,0,0
100003,Cabernet Sauvignon,3,Varietal/100%,['Cabernet Sauvignon'],"[Beef, Lamb, Poultry]",12.0,4,3,BR,Brazil,...,0,0,0,0,0,0,0,0,0,0
100005,Maison de Ville Cabernet-Merlot,3,Assemblage/Bordeaux Red Blend,"['Cabernet Sauvignon', 'Merlot']","[Beef, Lamb, Game Meat, Poultry]",11.0,4,2,BR,Brazil,...,0,0,0,0,0,0,0,0,0,0
100007,Do Lugar Moscatel Espumantes,4,Varietal/100%,['Muscat/Moscato Bianco'],"[Pork, Rich Fish, Shellfish]",7.5,3,3,BR,Brazil,...,0,1,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
199408,Petite Arvine,1,Varietal/100%,['Petite Arvine'],"[Pasta, Shellfish, Maturated Cheese, Hard Chee...",13.5,3,3,CH,Switzerland,...,0,1,0,0,0,0,0,0,0,0
199481,St. Laurent Reserve,3,Varietal/100%,['St. Laurent'],"[Beef, Lamb, Game Meat]",14.0,3,3,AT,Austria,...,0,0,0,0,0,0,0,0,0,0
199533,Mariengarten Chardonnay,1,Varietal/100%,['Chardonnay'],"[Pork, Rich Fish, Vegetarian, Poultry]",12.2,4,3,AT,Austria,...,0,0,0,0,0,0,0,0,0,1


In [75]:
#FOR WINE NAME

wines_df = wines_df.reset_index(drop=True)

def get_recommendations(wine_name, cosine_sim=cosine_sim):
    if wine_name not in wines_df['WineName'].values:
        return f"Wine '{wine_name}' not found. Please check the wine name."

    idx = wines_df[wines_df['WineName'] == wine_name].index[0]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[0:6]
    wine_indices = [i[0] for i in sim_scores]
    wine_similarities = [i[1] for i in sim_scores]
    
    recommendations = wines_df.iloc[wine_indices][['WineName', 'Type', 'Grapes', 'Body', 'Acidity', 'ABV', 'Harmonize']].copy()
    recommendations['Similarity'] = wine_similarities
    
    return recommendations



In [76]:
print(get_recommendations("Espumante Moscatel"))
#print(get_recommendations("Ancellotta"))


                         WineName  Type                      Grapes  Body  \
0              Espumante Moscatel     4          ['Muscat/Moscato']     3   
4    Do Lugar Moscatel Espumantes     4   ['Muscat/Moscato Bianco']     3   
28                       Moscatel     4          ['Muscat/Moscato']     3   
33   Terranova Espumante Moscatel     4          ['Muscat/Moscato']     3   
15               Malbec Brut Rosé     4                  ['Malbec']     3   
16  Espumante Método Charmat Brut     4  ['Chardonnay', 'Riesling']     3   

    Acidity   ABV                     Harmonize  Similarity  
0         3   7.5  [Pork, Rich Fish, Shellfish]         1.0  
4         3   7.5  [Pork, Rich Fish, Shellfish]         1.0  
28        3   7.5  [Pork, Rich Fish, Shellfish]         1.0  
33        3   7.5  [Pork, Rich Fish, Shellfish]         1.0  
15        3  11.5  [Pork, Rich Fish, Shellfish]         0.8  
16        3  12.0  [Pork, Rich Fish, Shellfish]         0.8  


In [77]:
random_user = np.random.choice(ratings_df['UserID'].unique())

user_ratings = ratings_df[ratings_df['UserID'] == random_user]
highest_rated_wine = user_ratings.loc[user_ratings['Rating'].idxmax()]

def get_wine_name(wine_id):
    wine_row = wines_df[wines_df['WineID'] == wine_id]
    if not wine_row.empty:
        return wine_row['WineName'].values[0]
    else:
        return f"Wine ID {wine_id} not found."


print(f"Random User ID: {random_user}")
print(f"Highest Rated Wine for User {random_user}:")
print(f"WineID: {highest_rated_wine['WineID']}, Rating: {highest_rated_wine['Rating']}")


wine_name = get_wine_name(highest_rated_wine['WineID'])
print(f"Wine Name: {wine_name}")
print(get_recommendations(wine_name))

Random User ID: 1224144
Highest Rated Wine for User 1224144:
WineID: 111417, Rating: 4.5
Wine Name: Pauillac (Grand Cru Classé)
                        WineName  Type  \
190  Pauillac (Grand Cru Classé)     3   
517           Camins del Priorat     3   
523                      Priorat     3   
559           Cabernet Sauvignon     3   
567           Cabernet Sauvignon     3   
649          Cocodrilo Red Blend     3   

                                                Grapes  Body  Acidity   ABV  \
190  ['Cabernet Sauvignon', 'Merlot', 'Cabernet Fra...     4        3  14.5   
517                                       ['Garnacha']     4        3  14.5   
523  ['Garnacha', 'Cabernet Sauvignon', 'Syrah/Shir...     4        3  14.5   
559                             ['Cabernet Sauvignon']     4        3  14.5   
567                             ['Cabernet Sauvignon']     4        3  14.5   
649  ['Cabernet Sauvignon', 'Malbec', 'Petit Verdot...     4        3  14.5   

                       